In [4]:
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw
import cv2
import numpy as np
import glob

#List tiff file
patchfile = glob.glob("*.tiff")

for impatch in patchfile:

    #Import image
    im = Image.open(impatch)

    #Set to array
    imarray = np.array(im)

    #Delete R B color
    gimarray = imarray.copy()
    gimarray[:, :, 0] = 0
    gimarray[:, :, 2] = 0

    #Fillter noise
    gimarray[gimarray < 30] = 0

    #Split G color array
    gareaimarray = gimarray[:, :, 1]
    cvgareaimarray = gareaimarray.copy() #Bufer for cv process
    cvgareaimarray[cvgareaimarray > 0] = 100 #Set to plain gray scale

    #Calculate intentsive of green area
    gareaimarray = gareaimarray[gareaimarray > 0]
    print(impatch)
    print(gareaimarray.mean())
      
    #Find contour
    _, contours, _ = cv2.findContours(cvgareaimarray, 
                                  cv2.RETR_EXTERNAL, 
                                  cv2.CHAIN_APPROX_SIMPLE)


    i = 0
    for c in contours:
       
        if cv2.contourArea(c) > 500:
            i = i + 1
            #Check center of contours
            M = cv2.moments(c)
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
    
            hull = cv2.convexHull(c)
            cv2.drawContours(imarray, [hull], 0, (0, 0, 255), 2)
            cv2.putText(imarray, str(i), (cX - 10, cY), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    #combine 2 image
    imarray = np.concatenate((imarray, np.full((1024, 2, 3), 255, dtype="uint8"), gimarray), axis=1)
      
    #Draw text in photo
    Finalim = Image.fromarray(imarray)
    draw = ImageDraw.Draw(Finalim)
    font = ImageFont.truetype('Montserrat-Black.ttf', 20)
    text = "Mean avg. : " + str(gareaimarray.mean()/3) + "\nTotal IntDen : " + str(gareaimarray.sum()/3)
    draw.text((1031, 5), text, (255,255,255), font=font)

    #Save image
    Finalim.save("Split" + impatch[:-5] + ".jpg")


Tol4-1 high.tiff
65.41690352335496
TolC251-1.tiff
52.72097145896118
